# mimi® Cloud APIを試す

本ページは読み取り専用のため、ご自身のGoogle Driveに一度コピーしてから実行してください。

※「ファイル」→「ドライブにコピー」を選択することで、自分のGoogle Driveに保存できます。

## 音声API使用のための事前準備

最初にmimi®のドキュメントにある、「[音声API使用のための事前準備](https://mimi.readme.io/docs/first-step)」ページの手順に従ってログインを行い、

- アプリケーションID
- クライアントID
- クライアントシークレット

を手元に控えてください。

（アクセストークンは、次セルにて実行＆取得可能です）

## アクセストークンを取得

まず、アクセストークンを取得するためのクエリパラメータを設定します。


アプリケーションIDとクライアントIDは、":"(colon)で連結して一つの文字列として設定してください。

例：fa92a3ca4fbc406b11f3b5c903bf914b:a4601265925a4f208a44a57fa5969cbd

クライアントシークレットは、そのまま一つの文字列として設定してください。特に、シークレットは他の人に教えたりしないようにしましょう。

↓の各変数の値を修正し、「[ ]」マークにマウスカーソルを当ててクリックすると、各セルのコードを実行することができます。読み進めながら順番に実行してみてください。

In [ ]:
# <アプリケーションID>:<クライアントID>の形でコロンつなぎで一つの文字列として設定してください
cli_id = '<アプリケーションID>:<クライアントID>'
# クライアントシークレットを文字列に設定してください (他の人に漏洩しないよう管理してください)
cli_secret = '<クライアントシークレット>'

設定が完了したら、次のコードを実行してアクセストークンを取得します。`token`という変数にアクセストークン文字列が格納され、以降の音声APIではこの値を使用します。

アクセストークンの有効期限は発行から1時間ですので、期限切れになったら（音声APIで401が返却されるようになったら）以下のコードを再度実行してアクセストークンを再取得してください。

In [ ]:
import requests
url = "https://auth.mimi.fd.ai/v2/token"

payload = {
    'client_id': cli_id,
    'client_secret': cli_secret,
    'grant_type': 'https://auth.mimi.fd.ai/grant_type/client_credentials',
    'scope': 'https://apis.mimi.fd.ai/auth/asr/websocket-api-service;https://apis.mimi.fd.ai/auth/asr/http-api-service;https://apis.mimi.fd.ai/auth/nict-asr/websocket-api-service;https://apis.mimi.fd.ai/auth/nict-asr/http-api-service;https://apis.mimi.fd.ai/auth/nict-tts/http-api-service;https://apis.mimi.fd.ai/auth/nict-tra/http-api-service',
}
resp = requests.post(url, data=payload)
if resp.status_code == 200:
  token = resp.json()['accessToken']
else:
  token = ''
token



## 音声認識を実行

今回は、HTTPにて音声ファイルを送信する方式を利用します。

WebSocketで、リアルタイムに音声を取得しながら分割転送する方式も可能ですが、お試し利用なので割愛します。詳細はmimi®のドキュメントの[WebSocket API仕様](https://mimi.readme.io/docs/asr-websocket-service)および[WebSocket APIの詳細動作ページ](https://mimi.readme.io/docs/mimi-websocket-api-spec)をご覧ください。

まず、音声ファイルを`audio.raw`というファイル名でアップロードします。

※←のメニューを開き、「ファイル」タブを選択、「アップロード」のメニューからアップロードしてください。


音声データは、以下の形式に変換したものをご利用ください。

- RAW形式：16kHz・16bit int（リトルエンディアン）・1channel

[ここ](https://github.com/FairyDevicesRD/libmimiio/blob/983c31e5060c5ce18fb3c4fa44170a3c15463dcc/examples/audio.raw)に、サンプルファイルもございますので用意が難しい場合はこちらをご利用ください。

In [ ]:
try:
  data = open('audio.raw', 'rb').read()
except FileNotFoundError:
  data = requests.get('https://github.com/FairyDevicesRD/libmimiio/raw/master/examples/audio.raw').content
  
data

mimi® ASR 版

In [ ]:
headers = {
    'Authorization': 'Bearer {}'.format(token),
    'Content-Type': 'audio/x-pcm;bit=16;rate=16000;channels=1',
    'x-mimi-input-language': 'ja',
    'x-mimi-process': 'asr',
}

url = 'https://service.mimi.fd.ai/'
resp = requests.post(url, headers=headers, data=data)
resp.json()


mimi® ASR powered by NICT 版

In [ ]:
headers = {
    'Authorization': 'Bearer {}'.format(token),
    'Content-Type': 'audio/x-pcm;bit=16;rate=16000;channels=1',
    'x-mimi-input-language': 'ja',
    'x-mimi-process': 'nict-asr',
}

url = 'https://service.mimi.fd.ai/'
resp = requests.post(url, headers=headers, data=data)
resp.json()


## 機械翻訳を実行

機械翻訳は、入力言語とそのテキスト、出力言語（翻訳対象の言語）を指定して、HTTP POSTにて実行します。

以下では、サンプルテキストを入力済みですが、もしできれば、みなさんの手で上記音声認識の結果から文章を抽出して入力テキストに設定し翻訳してみましょう。

In [ ]:
headers = {
    'Authorization': 'Bearer {}'.format(token),
}

data = {
    'source_lang': 'ja',
    'target_lang': 'en',
    'text': "ちょっと遅い昼食をとるためファミリーレストランに入ったのです。",
}

url = 'https://tra.mimi.fd.ai/machine_translation'
resp = requests.post(url, headers=headers, data=data)
resp.json()

## 音声合成を実行

音声合成は、入力言語およびテキストを指定し、HTTP POSTすることで実行します。

以下では、機械翻訳同様サンプルテキストを入力済みですが、上記機械翻訳結果を文章で抽出し、入力テキストに設定してみましょう。

このように、音声認識〜機械翻訳〜音声合成 を連携して実行させることで、音声翻訳機能を実現することができます。

In [ ]:
headers = {
    'Authorization': 'Bearer {}'.format(token),
}

data = {
    'lang': 'ja',
    'engine': 'nict',
    'text': 'ちょっと遅い昼食をとるためファミリーレストランに入ったのです。',
}

url = 'https://tts.mimi.fd.ai/speech_synthesis'
resp = requests.post(url, headers=headers, data=data)
if resp.status_code == 200:
  with open('resp.wav', 'wb') as fout:
    fout.write(resp.content)
    "speech synthesis succeeded."
else:
  resp.json()

取得した合成音声を再生するには以下を実行します。

In [ ]:
from IPython.display import Audio
Audio('resp.wav')

# おわりに

もし、音声認識・機械翻訳・音声合成の機能にご興味を持っていただけましたら、ぜひ弊社フェアリーデバイセズ株式会社へお問い合わせください。

コーポレートサイト：
http://www.fairydevices.jp/